**no care was taken to make sure this is theoretically/quantitatively correct** for a first pass. Produces uneven illumination (or uneven autofluorescence/background depending on how it's modified) and adds some shot-like noise on top of this to corrupt the signal. The goal is to simulate lower signal data, since our real starting sample data has high contrast and high SNR. 

In [97]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import seaborn as sns
import math
import scipy.misc
import random

sns.set_style('dark', rc={'image.cmap':'inferno'})

In [98]:
#parameters to be used later
illumination_unevenness = 0.8 #modifies the sigma determining the illumination artifact
illumination_unevenness_power = 4
mean_dark_current = 10**2 
percent_dead_pix = 0.1
bit_depth_save = 16

In [99]:
from skimage.io import imread
data_nodrug = imread("../data/confocal_drug_panel/drugA_raw.tif")

In [102]:
data_nodrug.dtype
data_nodrug.shape

im = data_nodrug[3,:,:,2]

plt.imshow(im, vmin = 0, vmax = im.max()*0.5)

In [103]:
mid_index = int(im.shape[0]/2)
print(mid_index)

In [104]:
#crudely simulate uneven illumination
mu = mid_index
sigma = mid_index*illumination_unevenness

def gaussian(x, mu, sigma):
    return np.exp(-np.power(x - mu, 2.) / (2 * np.power(sigma, 2.)))

gauss = gaussian(np.linspace(0, im.shape[0]-1, im.shape[1]), mu, sigma)

dim1 = im.shape[0]
dim2 = im.shape[1]
noiseD1 = np.zeros([dim1,dim2]) 

for x in range(0, dim1):
    noiseD1[:,x] = gauss

illumination = np.power(noiseD1.transpose()*noiseD1 , illumination_unevenness_power/2)

im_illum_artifact = illumination*im

plt.imshow(im_illum_artifact, vmin = 0, vmax = im_illum_artifact.max()*0.5)

In [105]:
#add dark/thermal noise, uniform across image and normally distributed
#not technically correct, noise should come from dark current shot noise
dark_current = np.random.normal(mean_dark_current, 1, [dim1,dim2])
noisy = im_illum_artifact + dark_current


In [106]:
#pretend this is a ccd or scmos and add some dead pixels
num_dead = int(percent_dead_pix*dim1*dim2/100)

idxs = random.sample(list(np.arange(dim1*dim2)),num_dead)
idx_bool = np.zeros([1,(dim1)*(dim2)], dtype=bool)
idx_bool[0,np.array(idxs)] = True
idx2d_bool = np.reshape(idx_bool, (dim1, dim2))

noisy[idx2d_bool] = 0

In [107]:
plt.imshow(noisy[300:500,300:500], vmin = 0, vmax = noisy.max*0.7())

In [108]:
noisy = noisy/noisy.max() * 2**bit_depth_save
mpimg.imsave("../data/uneven_illum_test.tif",noisy)

Make this one block of text so can loop through

In [ ]:
# import json
# with open('../data/confocal_drug_panel/drug.json', mode='r') as f_drug:
#     meta_drug = json.load(f_drug)
# with open('../data/no_drug.json', mode='r') as f_nodrug:
#     meta_nodrug = json.load(f_nodrug)

# drug_stack = {}
# nodrug_stack = {}
# for idx, channel in enumerate(meta_drug['channels']):
#     drug_stack[channel] = data_drug[:,:,idx]
#     nodrug_stack[channel] = data_nodrug[:,:,idx]    
#     print(channel)

illumination_unevenness = 0.8 #modifies the sigma determining the illumination artifact
illumination_unevenness_power = 4
mean_dark_current = 10**2 
percent_dead_pix = 0.1
bit_depth_save = 16
save_fame = "drugA.tif"
from skimage.io import imread
data_nodrug = imread("../data/confocal_drug_panel/drugA_raw.tif")

im = data_nodrug[3,:,:,2]
plt.imshow(im, vmin = 0, vmax = im.max()*0.5)


def add_noise(im, illumination_unevenness, illumination_unevenness_power, mean_dark_current, percent_dead_pix, save_fname, bit_depth_save)
    mid_index = int(im.shape[0]/2)
    #crudely simulate uneven illumination
    mu = mid_index
    sigma = mid_index*illumination_unevenness
    def gaussian(x, mu, sigma):
        return np.exp(-np.power(x - mu, 2.) / (2 * np.power(sigma, 2.)))
    gauss = gaussian(np.linspace(0, im.shape[0]-1, im.shape[1]), mu, sigma)
    dim1 = im.shape[0]
    dim2 = im.shape[1]
    noiseD1 = np.zeros([dim1,dim2]) 

    for x in range(0, dim1):
        noiseD1[:,x] = gauss

    illumination = np.power(noiseD1.transpose()*noiseD1 , illumination_unevenness_power/2)
    im_illum_artifact = illumination*im
    plt.imshow(im_illum_artifact, vmin = 0, vmax = im_illum_artifact.max()*0.5)

    #add dark/thermal noise, uniform across image and normally distributed
    #not technically correct, noise should come from dark current shot noise
    dark_current = np.random.normal(mean_dark_current, 1, [dim1,dim2])
    noisy = im_illum_artifact + dark_current

    #pretend this is a ccd or scmos and add some dead pixels
    num_dead = int(percent_dead_pix*dim1*dim2/100)

    #the below line doesn't run, and I'm not able to trouble-shoot it right now
    noisy[random.sample(int(np.arange(dim1)),num_dead) , random.sample(int(np.arange(dim2)), num_dead)] = 0

    noisy = noisy/noisy.max() * 2**bit_depth_save
    return noisy

